## Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, BayesianRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime

DATEPARSE = "%Y-%m-%d %H:%M:%S"

# Read data
### Add data into /taxi_data/
### Github stores a sample file of this. Download the rest and do the same.

In [2]:
def time_to_minutes(_today):
    return (_today.weekday()+1) * 24*60 + (_today.hour*60) + (_today.minute)
def add_zero_padding(number):
    if len(str(number)) == 1: return "0" + str(number)
    else: return number
def format_df(DF, pickup_date_column, dropof_date_column):
    for i, x in DF.iterrows():
        DF.at[i,"month"] = 1
        start = datetime.strptime(x[pickup_date_column], DATEPARSE)
        stop = datetime.strptime(x[dropof_date_column], DATEPARSE)

        DF.at[i, pickup_date_column] = time_to_minutes(start)
        DF.at[i, dropof_date_column] = time_to_minutes(stop)
    return DF

In [3]:
#GREEN

header = True
for x in range(1, 13): # this takes 10-15 min
    try:
        dfGreen = pd.read_csv("taxi_data\green_tripdata_2019-" + str(add_zero_padding(x)) + ".csv", usecols=["PULocationID", "DOLocationID", "lpep_pickup_datetime", "lpep_dropoff_datetime"])
        dfGreen = format_df(dfGreen, "lpep_pickup_datetime", "lpep_dropoff_datetime")
        dfGreen.to_csv("taxi_data\green_tripdata_2019_concat.csv", mode='a+', header=header, index=False)
        header=False
        print("GREEN: " + str(x))
    except Exception as f:
        print(f)

"""
# YELLOW
header = True
for x in range(1, 13):
    try:
        dfYellow = pd.read_csv("taxi_data\yellow_tripdata_2019-" + str(add_zero_padding(x)) + ".csv")
        dfYellow = format_df(dfYellow, "tpep_pickup_datetime", "tpep_dropoff_datetime")
        dfYellow.to_csv("taxi_data\yellow_tripdata_2019_concat.csv", mode='a+', header=header, index=False)
        header=False
        print("YELLOW: " + str(x))
    except Exception as f:
        print(f)
"""

GREEN: 1
GREEN: 2
[Errno 2] No such file or directory: 'taxi_data\\green_tripdata_2019-03.csv'
GREEN: 4
GREEN: 5
GREEN: 6
GREEN: 7
GREEN: 8
GREEN: 9
GREEN: 10
GREEN: 11
GREEN: 12


'\n# YELLOW\nheader = True\nfor x in range(1, 13):\n    try:\n        dfYellow = pd.read_csv("taxi_data\\yellow_tripdata_2019-" + str(add_zero_padding(x)) + ".csv")\n        dfYellow = format_df(dfYellow, "tpep_pickup_datetime", "tpep_dropoff_datetime")\n        dfYellow.to_csv("taxi_data\\yellow_tripdata_2019_concat.csv", mode=\'a+\', header=header, index=False)\n        header=False\n        print("YELLOW: " + str(x))\n    except Exception as f:\n        print(f)\n'

# Train

In [5]:
dfGreen = pd.read_csv("taxi_data\green_tripdata_2019_concat.csv", chunksize=100000)

model = KNeighborsClassifier(3)

first = True
for chunk in dfGreen:
    X_green = chunk[["PULocationID", "DOLocationID","lpep_pickup_datetime", "month"]] # input cols
    y_green = chunk[["lpep_dropoff_datetime"]] # output cols
    
    X_train, X_test, y_train, y_test = train_test_split(X_green, y_green, test_size = 0.25)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
    print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))
    
    break
model.predict([])

C:\Python310\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


       PULocationID  DOLocationID  lpep_pickup_datetime  month
97376           255            80                 10221    1.0
44050           169           244                  6624    1.0
16199            49           197                  4715    1.0
39197           181           195                  6400    1.0
71846            52            85                  8419    1.0
...             ...           ...                   ...    ...
86908            74           244                  9582    1.0
49914            75            24                  6900    1.0
11826            82            82                  4010    1.0
47841           247            69                  6787    1.0
93683            74           237                  9933    1.0

[25000 rows x 4 columns]
Mean squared error: 66836.85
Coefficient of determination: 0.99
